In [ ]:
##https://www.kaggle.com/jiweiliu/lgb-2-leaves-augment
##https://www.kaggle.com/yag320/list-of-fake-samples-and-public-private-lb-split
##https://www.kaggle.com/qitian51212/simple-magic-var-0-922

In [48]:
# LOAD LIBRARIES
#import matplotlib.pyplot as plt
#import seaborn as sns
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import statsmodels.api as sm
from tqdm import tnrange, tqdm, tqdm_notebook
from sklearn.model_selection import KFold, StratifiedKFold

In [50]:
# GET INDICIES OF REAL TEST DATA
#######################
# TAKE FROM YAG320'S KERNEL
# https://www.kaggle.com/yag320/list-of-fake-samples-and-public-private-lb-split

df_test = pd.read_csv('input/test_santander.csv')
df_test.drop(['ID_code'], axis=1, inplace=True)
df_test = df_test.values

unique_samples = []
unique_count = np.zeros_like(df_test)
for feature in range(df_test.shape[1]):
    _, index_, count_ = np.unique(df_test[:, feature], return_counts=True, return_index=True)
    unique_count[index_[count_ == 1], feature] += 1

# Samples which have unique values are real the others are fake
real_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) > 0)[:, 0]
synthetic_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) == 0)[:, 0]

print('Found',len(real_samples_indexes),'real test')
print('Found',len(synthetic_samples_indexes),'fake test')
#######################

Found 100000 real test
Found 100000 fake test


In [51]:
#Load Datas
#df_testをreal_testで再使用するのでもう一回load
df_train = pd.read_csv('input/train_santander.csv')
df_test = pd.read_csv('input/test_santander.csv')

In [62]:
print(df_train.shape)
print(df_test.shape)

(200000, 202)
(200000, 201)


In [52]:
#Check data
df_train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [53]:
df_test.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [54]:
real_ids = list(set(real_samples_indexes))
real_test = df_test.iloc[real_ids]

In [58]:
##https://www.kaggle.com/qitian51212/simple-magic-var-0-922
features = [col for col in df_train.columns if col not in ['ID_code', 'target']]

In [59]:
print(len(features))

200


In [61]:
#concatenate train data and real test data with new features
df = pd.concat([df_train,real_test], axis = 0)

for feat in tqdm_notebook(features):
    df[feat+'_var'] = df.groupby([feat])[feat].transform('var')

for feat in tqdm_notebook(features):
    df[feat+'plus_'] = df[feat] + df[feat+'_var']

drop_features = [c for c in df.columns if '_var' in c]
df.drop(drop_features, axis=1, inplace=True)

In [63]:
#divide two datas
df_train = df.iloc[:df_train.shape[0]]
df_test2 = df.iloc[df_train.shape[0]:]

In [64]:
#新trainデータのtargetを使用してfeaturesを再定義
features = [col for col in df_train.columns if col not in ['ID_code', 'target']]
target = df_train['target']

In [65]:
print(df_train.shape)
print(df_test2.shape)
print(len(features))

(200000, 402)
(100000, 402)
400


In [66]:
df_train.head()

,ID_code,target,var_0,var_1,var_10,var_100,var_101,var_102,var_103,var_104,...,var_190plus_,var_191plus_,var_192plus_,var_193plus_,var_194plus_,var_195plus_,var_196plus_,var_197plus_,var_198plus_,var_199plus_
0,train_0,0.0,8.9255,-6.7863,2.9252,9.4763,13.3102,26.5376,1.4403,14.7100,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0.0,11.5006,-4.1473,-0.4032,-13.6950,8.4068,35.4734,1.7093,15.1866,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0.0,8.6093,-2.7457,-0.3249,-0.3939,12.6317,14.8863,1.3854,15.0284,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0.0,11.0604,-2.1518,2.3061,-19.8592,22.5316,18.6129,1.3512,9.3291,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0.0,9.8369,-1.4834,-9.4458,-22.9264,12.3562,17.3410,1.6940,7.1179,...,-1.4905,9.5214,NaN,NaN,NaN,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [67]:
df_test2.head()

,ID_code,target,var_0,var_1,var_10,var_100,var_101,var_102,var_103,var_104,...,var_190plus_,var_191plus_,var_192plus_,var_193plus_,var_194plus_,var_195plus_,var_196plus_,var_197plus_,var_198plus_,var_199plus_
3,test_3,NaN,8.5374,-1.3222,0.0095,1.7021,2.5363,3.8763,1.5173,13.4083,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,NaN
7,test_7,NaN,17.3035,-2.4212,4.0355,-22.7363,11.6984,18.0784,1.7108,9.2897,...,4.4676,4.4214,0.9303,1.4994,15.2648,-1.7931,6.5316,10.4855,NaN,0.7283
11,test_11,NaN,10.6137,-2.1898,2.4620,-14.9298,23.1641,5.2171,1.7760,8.4856,...,13.1683,4.0625,-0.1537,7.9787,18.4518,0.1000,NaN,9.2355,15.0721,NaN
15,test_15,NaN,14.8595,-4.5378,3.9229,-13.1135,19.4192,21.7291,1.3708,8.5129,...,2.6735,5.8526,NaN,2.5020,22.8224,-0.9325,8.6849,10.2848,17.4932,NaN
16,test_16,NaN,14.1732,-5.1490,-5.8456,-2.4555,12.4817,17.8563,1.6619,12.7459,...,0.8640,5.9058,1.3140,4.8961,20.1087,1.1051,7.7184,9.3406,21.1746,-2.0098


In [68]:
#LightGBM parameters
#Parameters are referenced by https://www.kaggle.com/jiweiliu/lgb-2-leaves-augment

lgb_params = {
    "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : -1,
    "num_leaves" : 13,
    "learning_rate" : 0.01,
    "bagging_freq": 5,
    "bagging_fraction" : 0.4,
    "feature_fraction" : 0.05,
    "min_data_in_leaf": 80,
    "min_sum_heassian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    #"lambda_l1" : 5,
    #"lambda_l2" : 5,
    "bagging_seed" : random_state,
    "verbosity" : 1,
    "seed": random_state
}

In [69]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=4242)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test2))
val_aucs = []
feature_importance_df = pd.DataFrame()

In [70]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=target.iloc[val_idx])
    clf = lgb.train(lgb_params, trn_data, 100000, valid_sets = [trn_data, val_data], verbose_eval=5000, early_stopping_rounds = 3000)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    val_aucs.append(roc_auc_score(target[val_idx] , oof[val_idx] ))
    predictions += clf.predict(df_test2[features], num_iteration=clf.best_iteration) / folds.n_splits
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

Fold 0
Training until validation scores don't improve for 3000 rounds.
[5000]	training's auc: 0.950857	valid_1's auc: 0.92351
[10000]	training's auc: 0.970618	valid_1's auc: 0.924955


KeyboardInterrupt: 

In [20]:
mean_auc = np.mean(val_aucs)
std_auc = np.std(val_aucs)
all_auc = roc_auc_score(target, oof)
print("Mean auc: %.9f, std: %.9f. All auc: %.9f." % (mean_auc, std_auc, all_auc))

Mean auc: 0.922200288, std: 0.001920134. All auc: 0.922050274.


In [21]:
subreal = pd.DataFrame({"ID_code":df_test2["ID_code"].values})
subreal['target']=predictions
sub = pd.DataFrame({"ID_code": df_test.ID_code.values})

In [23]:
finalsub = sub.set_index('ID_code').join(subreal.set_index('ID_code')).reset_index()
finalsub.fillna(0,inplace=True)
finalsub.to_csv("output/submission.csv", index=False)